<a href="https://colab.research.google.com/github/MWFK/Minervas/blob/master/Search_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
!pip install pymupdf pandas

In [25]:
import os
import openai
from openai import OpenAI
import json
from google.colab import drive
import fitz  # PyMuPDF
import pandas as pd
import numpy as np

In [6]:
drive.mount('/content/drive')

# Load API key from configuration file
config_path = '/content/drive/MyDrive/Colab_Notebooks/API_Keys/openai_api_key.json'  # Update with your path
with open(config_path, 'r') as f:
    config = json.load(f)

api_key = config.get('api_key', None)

if api_key is None:
    raise ValueError("API key not found in the configuration file.")

print("API key loaded successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
API key loaded successfully.


In [12]:
os.environ["OPENAI_API_KEY"] = api_key
print(openai.__version__)
client = OpenAI()

1.10.0


In [15]:
def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    data = []

    for page_number in range(doc.page_count):
        page = doc[page_number]
        text = page.get_text()

        data.append({
            'PageNumber': page_number + 1,
            'Text': text
        })

    doc.close()
    return data

# Replace 'your_pdf_file.pdf' with the actual PDF file path
pdf_path = '/content/AIF.pdf'

# Convert PDF to DataFrame
pdf_data = pdf_to_text(pdf_path)
df = pd.DataFrame(pdf_data)

# Display the DataFrame
df

,PageNumber,Text
0,1,Oxford Artificial Intelligence in Fintech and ...
1,2,Oxford Artificial Intelligence in Fintech and ...
2,3,Oxford Artificial Intelligence in Fintech and ...
3,4,Oxford Artificial Intelligence in Fintech and ...
4,5,Oxford Artificial Intelligence in Fintech and ...
...,...,...
331,332,13\nOxford Artificial Intelligence in Fintech ...
332,333,14\nOxford Artificial Intelligence in Fintech ...
333,334,15\nOxford Artificial Intelligence in Fintech ...
334,335,16\nOxford Artificial Intelligence in Fintech ...


In [16]:
# Export DataFrame to CSV
csv_file_path = '/content/AIF.csv'
df.to_csv(csv_file_path, index=False)

print(f'DataFrame exported to {csv_file_path}')

DataFrame exported to /content/AIF.csv


In [20]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['Text_embedding'] = df.Text.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('AIF_embedded.csv', index=False)

In [21]:
df.iloc[47:55]

,PageNumber,Text,ada_embedding
47,48,Oxford Artificial Intelligence in Fintech and ...,"[-0.017736369743943214, 0.04117727279663086, 0..."
48,49,Oxford Artificial Intelligence in Fintech and ...,"[-0.013243403285741806, 0.037315741181373596, ..."
49,50,Oxford Artificial Intelligence in Fintech and ...,"[-0.015288255177438259, 0.006591282784938812, ..."
50,51,Oxford Artificial Intelligence in Fintech and ...,"[0.01251201331615448, -0.014181532897055149, 0..."
51,52,Oxford Artificial Intelligence in Fintech and ...,"[-0.02634011022746563, -0.00967896543443203, 0..."
52,53,Oxford Artificial Intelligence in Fintech and ...,"[-0.028539549559354782, -0.006737750489264727,..."
53,54,Oxford Artificial Intelligence in Fintech and ...,"[0.005471996963024139, 0.018172774463891983, 0..."
54,55,Oxford Artificial Intelligence in Fintech and ...,"[-0.006925802677869797, -0.011107225902378559,..."


In [23]:
footer_1='Oxford Artificial Intelligence in Fintech and Open Banking Programme'
footer_2='oxfordsuccess@esmelearning.com'
footer_3='© 2022 University of Oxford. All rights reserved. Any other company names and trademarks are the property of their respective owners.'

phrases_to_delete = [footer_1,footer_2,footer_3,'\n']

# Apply str.replace() for each phrase in the list to the 'Text' column
for phrase in phrases_to_delete:
    df['Text'] = df['Text'].str.replace(phrase, '')
df

<ipython-input-23-183cb1486d72>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace(phrase, '')


,PageNumber,Text,ada_embedding
0,1,| 1Orientation2022© 2022 University of Oxfor...,"[-0.008428266271948814, 0.00658399285748601, 0..."
1,2,| 2Table Of ContentsOxford AI in Fintech and...,"[-0.01903080940246582, 0.028201499953866005, 0..."
2,3,| 31.0 Oxford AI in Fintech and Open Banking...,"[-0.009327526204288006, 0.010276480577886105, ..."
3,4,| 4Who Should Take This ProgrammeThis progra...,"[-0.01556289941072464, 0.013242220506072044, 0..."
4,5,| 5About the Curriculum The Oxford AI in Fi...,"[-0.013321353122591972, 0.03288555517792702, 0..."
...,...,...,...
331,332,13 artificial neural network - See neural ne...,"[-0.0030018428806215525, -0.01519651897251606,..."
332,333,14 Competition & Markets Authority (CMA) - T...,"[0.02134447731077671, 0.018453774973750114, 0...."
333,334,15 European Banking Authority (EBA) - Accord...,"[-0.009156021289527416, 0.012509927153587341, ..."
334,335,16 ML - See machine learning (ML).neural net...,"[0.0004295336257200688, -0.0031878002919256687..."


In [24]:
df.iloc[47:55]

,PageNumber,Text,ada_embedding
47,48,"4ObjectivesAt the end of this module, you w...","[-0.017736369743943214, 0.04117727279663086, 0..."
48,49,5Vocabulary CheckThe terms below are indust...,"[-0.013243403285741806, 0.037315741181373596, ..."
49,50,61.2 Introduction to AI1.2.1 The Data Busin...,"[-0.015288255177438259, 0.006591282784938812, ..."
50,51,"7(Alpeyev & Hyuga, 2020). Alibaba would go ...","[0.01251201331615448, -0.014181532897055149, 0..."
51,52,8a factory that bypasses humans completely....,"[-0.02634011022746563, -0.00967896543443203, 0..."
52,53,9And that distinction is very important bec...,"[-0.028539549559354782, -0.006737750489264727,..."
53,54,10the code block and click the Run button. ...,"[0.005471996963024139, 0.018172774463891983, 0..."
54,55,"11value after the period, such as ‘y = cred...","[-0.006925802677869797, -0.011107225902378559,..."


In [ ]:
# # To load the data from a saved file, you can run the following:
# df = pd.read_csv('AIF_embedded.csv')
# df['Text_embedding'] = df.ada_embedding.apply(eval).apply(np.array)

In [ ]:
# https://platform.openai.com/docs/guides/embeddings/use-cases
# https://cookbook.openai.com/examples/question_answering_using_embeddings